# Mesh generation for tutorial 06, case 1b

This file generates the mesh which is used in the following examples:
* 1b_poisson

The test case is from section 5.1 of
```
F. Negri, G. Rozza, A. Manzoni and A. Quarteroni. Reduced Basis Method for Parametrized Elliptic Optimal Control Problems. SIAM Journal on Scientific Computing, 35(5): A2316-A2340, 2013.
```

In [ ]:
import pygmsh
from dolfinx import Mesh, MeshFunction, MeshValueCollection, MPI
from dolfinx.cpp.mesh import CellType, GhostMode
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Geometrical parameters and related quantities

In [ ]:
L1 = 1.0
L2 = 3.0
H = 1.0
lcar = 0.05

### Create mesh

In [ ]:
geom = pygmsh.built_in.Geometry()
p0 = geom.add_point([0.0, 0.0, 0.0], lcar)
p1 = geom.add_point([L1, 0.0, 0.0], lcar)
p2 = geom.add_point([L1 + L2, 0.0, 0.0], lcar)
p3 = geom.add_point([L1 + L2, H, 0.0], lcar)
p4 = geom.add_point([L1, H, 0.0], lcar)
p5 = geom.add_point([0.0, H, 0.0], lcar)
l0 = geom.add_line(p0, p1)
l1 = geom.add_line(p1, p4)
l2 = geom.add_line(p4, p5)
l3 = geom.add_line(p5, p0)
l4 = geom.add_line(p1, p2)
l5 = geom.add_line(p2, p3)
l6 = geom.add_line(p3, p4)
geom.add_physical([l0, l4, l5, l6, l2, l3], label=1)
line_loop_rectangle_left = geom.add_line_loop([l0, l1, l2, l3])
line_loop_rectangle_right = geom.add_line_loop([l4, l5, l6, -l1])
rectangle_left = geom.add_plane_surface(line_loop_rectangle_left)
rectangle_right = geom.add_plane_surface(line_loop_rectangle_right)
geom.add_physical(rectangle_left, label=11)
geom.add_physical(rectangle_right, label=12)
pygmsh_mesh = pygmsh.generate_mesh(geom)
mesh = Mesh(MPI.comm_world, CellType.triangle, pygmsh_mesh.points[:, :2],
            pygmsh_mesh.cells_dict["triangle"], [], GhostMode.none)

In [ ]:
plot(mesh)

### Create subdomains

In [ ]:
subdomains_mvc = MeshValueCollection("size_t", mesh, mesh.topology.dim, pygmsh_mesh.cells_dict["triangle"],
                                     pygmsh_mesh.cell_data_dict["gmsh:physical"]["triangle"] - 10)
subdomains = MeshFunction("size_t", mesh, subdomains_mvc, 0)

### Create boundaries

In [ ]:
boundaries_mvc = MeshValueCollection("size_t", mesh, mesh.topology.dim - 1, pygmsh_mesh.cells_dict["line"],
                                     pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"])
boundaries = MeshFunction("size_t", mesh, boundaries_mvc, 0)

### Save

In [ ]:
with XDMFFile(MPI.comm_world, "rectangle.xdmf") as output:
    output.write(mesh)
with XDMFFile(MPI.comm_world, "rectangle_subdomains.xdmf") as output:
    output.write(subdomains)
with XDMFFile(MPI.comm_world, "rectangle_boundaries.xdmf") as output:
    output.write(boundaries)